In this tutorial we will be illustrating how to set up and use (kNN) Topological PCA for Single Cell RNA-Seq Data Analysis. We will first define all the steps behind tPCA, and then illustrate its use for cell type identification by clustering. The mathematical formulation of Topological PCA is 

$\min_{U,Q}\|X - UQ^T\|_{2,1} + \beta\|Q\|_{2,1} + \gamma \text{Tr}(Q^T(L_{\rm P})Q), \quad \text{s.t. } Q^TQ = I_m$

The first term, $\min_{U,Q}\|X - UQ^T\|_{2,1}$ is simply a robust variation of principal component analysis, where we are trying to approximate the gene expression matrix $X$ by low rank matrices $U$ and $Q$. 

The second term $\beta\|Q\|_{2,1}$ tries to enforce that the resulting embedded expression profiles are sparse, and this makes the result more interpretable due to the high sparsity of true gene expression data. $\beta$ is a parameter that controls how sparse we want the solution to be. 

The last term $\gamma \text{Tr}(Q^T(L_{\rm P})Q)$ is a graph embedding term, where we are seeking to preserve the cell-cell expression profile correlation relations described in the graph Laplacian term $L_p$. The idea is that if two cells have correlated expression profiles (and so are connected in the graph), then their embeddings should be located close to one another. 

The last condition, $Q^TQ = I_m$ is a constraint stating that $Q$ should be an orthogonal matrix. 

In [1]:
import numpy as np 
import scanpy as sc
from mclustpy import mclustpy

def gaussian_kernel(dist, t):
    '''
    gaussian kernel function for weighted edges
    '''
    return np.exp(-(dist**2 / t))

def Eu_dis(x):
    """
    Calculate the distance among each raw of x
    :param x: N X D
                N: number of samples
                D: Dimension of the feature
    :return: N X N distance matrix
    """
    x = np.asarray(x)
    aa = np.sum(np.multiply(x, x), 1)
    ab = x @ x.T
    dist_mat = aa + aa.T - 2 * ab
    dist_mat[dist_mat < 0] = 0
    dist_mat = np.sqrt(dist_mat)
    dist_mat = np.maximum(dist_mat, dist_mat.T)
    dist_mat = np.asarray(dist_mat)
    return dist_mat

def cal_weighted_adj(data, n_neighbors, t):
    '''
    Calculate weighted adjacency matrix based on KNN
    For each row of X, put an edge between nodes i and j
    If nodes are among the n_neighbors nearest neighbors of each other
    according to Euclidean distance
    '''
    dist = Eu_dis(data)
    n = dist.shape[0]
    gk_dist = gaussian_kernel(dist, t)
    W_L = np.zeros((n, n))
    for i in range(n):
        index_L = np.argsort(dist[i, :])[1:1 + n_neighbors] 
        len_index_L = len(index_L)
        for j in range(len_index_L):
            W_L[i, index_L[j]] = gk_dist[i, index_L[j]] #weighted edges
    W_L = np.maximum(W_L, W_L.T)
    return W_L

def cal_unweighted_adj(data, n_neighbors):
    '''
    Calculate unweighted adjacency matrix based on KNN
    '''
    dist = Eu_dis(data)
    n = dist.shape[0]
    W_L = np.zeros((n, n))
    for i in range(n):
        index_L = np.argsort(dist[i, :])[1:1 + n_neighbors]
        len_index_L = len(index_L)
        for j in range(len_index_L):
            W_L[i, index_L[j]] = 1 #edges not weighted
    W_L = np.maximum(W_L, W_L.T)
    return W_L

def cal_laplace(adj):
    N = adj.shape[0]
    D = np.zeros_like(adj)
    for i in range(N):
        D[i, i] = np.sum(adj[i]) # Degree Matrix
    L = D - adj  # Laplacian
    return L


def tPCA_Algorithm(xMat,laplace,beta,gamma,k,n):
    '''
    Optimization Algorithm of tPCA 
    Solve approximately via ADMM
    Need to compute optimal principal directions matrix U
    Projected Data matrix Q
    Error term matrix E = X - UQ^T
    Z matrix used to solve Q (see supplementary information)

    Inputs are data matrix X, laplacian, scale parameters, 
    number of reduced dimensions, number of original dimensions
    '''
    # Initialize thresholds, matrices
    obj1 = 0
    obj2 = 0
    thresh = 1e-50
    V = np.eye(n) 
    vMat = np.asarray(V) # Auxillary matrix to optimize L2,1 norm
    E = np.ones((xMat.shape[0],xMat.shape[1]))
    E = np.asarray(E) # Error term X - UQ^T
    C = np.ones((xMat.shape[0],xMat.shape[1]))
    C = np.asarray(C) # Lagrangian Multiplier
    laplace = np.asarray(laplace) #Lplacian
    miu = 1 #Penalty Term
    for m in range(0, 30):
        Z = (-(miu/2) * ((E - xMat + C/miu).T @ (E - xMat + C/miu))) + beta * vMat + gamma * laplace
        # cal Q (Projected Data Matrix)
        Z_eigVals, Z_eigVects = np.linalg.eig(np.asarray(Z))
        eigValIndice = np.argsort(Z_eigVals)
        n_eigValIndice = eigValIndice[0:k]
        n_Z_eigVect = Z_eigVects[:, n_eigValIndice]
        # Optimal Q given by eigenvectors corresponding
        # to smallest k eigenvectors
        Q = np.array(n_Z_eigVect)  
        # cal V 
        q = np.linalg.norm(Q, ord=2, axis=1)
        qq = 1.0 / (q * 2)
        VV = np.diag(qq)
        vMat = np.asarray(VV)
        qMat = np.asarray(Q)
        # cal U (Principal Directions)
        U = (xMat - E - C/miu) @ qMat
        # cal P (intermediate step)
        P = xMat - U @ qMat.T - C/miu
        # cal E (Error Term)
        for i in range(E.shape[1]):
            E[:,i] = (np.max((1 - 1.0 / (miu * np.linalg.norm(P[:,i]))),0)) * P[:,i]
        # update C 
        C = C + miu * (E - xMat + U @ qMat.T)
        # update miu
        miu = 1.2 * miu

        obj1 = np.linalg.norm(qMat)
        if m > 0:
            diff = obj2 - obj1
            if diff < thresh:
                break # end iterations if error within accepted threshold
        obj2 = obj1
    return U #return principal directions matrix


We then want to introduce filtration. The idea here is that different scales of gene expression correlations may correspond to different scales of cell interactions which we want to incorporate into the model. Alternatively, we may view this objective function as trying to preserve the intrinsic geometry of the data (the graph relations). Filtration allows us to try to preserve that geometry as viewed over multiple scales, making the model more robust. 

We can do this by including several steps of a filtration, where at each step we change the number of neighbors we are using to construct our $kNN$ cell graph. This allows us to formulate the $k$th graph Laplacian

$ L^k = (l_{ij}^k)\text{, } l_{ij}^k = 
    \begin{cases}
    -1,\text{ if } i \neq j \text{ and }\mathbf{x}_j \in \mathcal{N}_k(\mathbf{x}_i) \\
    0, \text{ otherwise}
    \end{cases}$

where $l_{ii}^k = -\sum_{j=1}^nl_{ij}^k$

We can then construct what we call an Accumulated Spectral Graph where we weight each Laplacian by some value $\zeta$ and sum them into a single term denoted $L_{\rm P}$. 

$L_{\rm P} := \sum_{k=1}^p\zeta_kL^k$

In [2]:

def cal_persistence_KNN(data, n_filtrations, zetas):
    n = data.shape[0]
    '''
    Consider n neighbors and reduce by 2 neighbors at
    each iteration of filtration down to 6 nearest neighbors
    (4 filtrations)
    '''
    num_neighbors_list = range(6, n_filtrations + 1, 3)
    num_filtrations = len(num_neighbors_list)

    PL = np.zeros((num_filtrations, n, n))
    zetas = np.array(zetas)

    for idx, num_neighbors in enumerate(num_neighbors_list):
        A = cal_unweighted_adj(data, num_neighbors)
        #print('num neighbors:', num_neighbors)
        PL[idx, :, :] = cal_laplace(A)
        #print("i'th PL:", PL[idx, :, :])
        #print("zeta_i:", zetas[idx])

    Persistent_Laplacian = np.sum(zetas[:, np.newaxis, np.newaxis] * PL, axis=0)
    return Persistent_Laplacian


Now we fit this model to the gene expression data at some combination of scales (some $\{ \zeta_i \}$ combination in the $PL$ term) We use $\zeta$ values of either 0 or 1 which corresponds to either 'turning on or off' that scale of graph connectivity in the graph embedding. Note that as the orthogonality constraint is no longer on $U$, we actually transform $X$ by the pseudo-inverse of $U$. 

In [3]:
def tPCA_cal_projections_KNN(X_data, beta1, gamma1, k_d, num_filtrations, zeta):
    n = len(X_data)  
    M = cal_persistence_KNN(X_data, num_filtrations, zeta)
    Y = tPCA_Algorithm(X_data.transpose(), M, beta1, gamma1, k_d, n)
    return Y

#Embed gene expression data
def tPCA_embedding(X, beta, gamma, k, zeta):
    #Principal Components
    PDM = tPCA_cal_projections_KNN(X, beta, gamma, k, 15, zeta)
    PDM = np.asarray(PDM)
    #print(PDM.shape)
    TM = ((np.linalg.inv(PDM.T @ PDM)) @ (PDM.T)).T
    #Projected Data Matrix
    Q = (X @ TM)
    #print(Q.shape)
    return Q

In [5]:
adata = sc.read_h5ad('/mnt/home/cottre61/GFP-GAT/STAGATE_pyG/GraphTransformer/Data/GSE67835.h5ad')
adata.var_names_make_unique()
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata = adata[:, adata.var['highly_variable']]

In [6]:
gamma = 1e2 # graph strength
beta = 1e1 # sparsity
k = 20 # embedding dim
zeta = [1,0,1,0] # connectivity scales to consider (corresponding to k = 6,8,10,12)

X = adata.X.toarray()

In [7]:
Q = tPCA_embedding(X, beta,gamma,k,zeta)
res = mclustpy(Q, G=7, modelNames='EEE', random_seed=2020)

R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.0.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


Alternatively we can cluster over multiple different scales, and then utilize those multiple clusterings to define a consensus clustering. We will explore this idea in MCIST. 